<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Stop Untagged EC2 Instances using unSkript legos.</b>
</div>

<br>

<center><h2>Stop Untagged EC2 Instances</h2></center>

# Steps Overview
    1) Filter all the instances which dont have any tags.
    2) Stop the instances after the we get through first step

Here we will use unSkript Filter AWS EC2 Untagged Instances Lego. This lego takes region as input. This inputs is used to findout all the instances which don't have tags.

In [5]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint
from beartype import beartype

@beartype
def aws_filter_untagged_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


def aws_filter_untagged_ec2_instances(handle, region: str) -> List:
    """aws_filter_untagged_ec2_instances Returns an array of instances which has no tags.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: str
        :param region: Region to filter instances.

        :rtype: Result of the API in the List form.
    """

    ec2Client = handle.client('ec2', region_name=region)

    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    # get the number of objects
    result = []
    for reservation in res:
        for instance in reservation['Instances']:
            try:
                tagged_instance = instance['Tags']
                if len(tagged_instance) == 0:
                    result.append(instance['InstanceId'])
            except Exception as e:
                result.append(instance['InstanceId'])

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')
task.configure(outputName="UntaggedInstances")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_untagged_ec2_instances, lego_printer=aws_filter_untagged_ec2_instances_printer, hdl=hdl, args=args)

Here we will use unSkript Stop AWS Instances Lego. This lego takes instance_id and region as input. This inputs is used to stop the instances which don't have tags.

In [3]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_stop_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_stop_instances(handle, instance_id: str, region: str) -> Dict:
    """aws_stop_instances Stops instances.

        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.

        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.

        :type inputParamsJson: string
        :param inputParamsJson: Json string of the input params.

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    output = {}
    res = ec2Client.stop_instances(InstanceIds=[instance_id])
    for instances in res['StoppingInstances']:
        output[instances['InstanceId']] = instances['CurrentState']

    return output


task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "instance_id": "iter_item",
    "region": "Region"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "UntaggedInstances",
    "iter_parameter": "instance_id"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(UntaggedInstances) > 0",
    "condition_result": true
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_stop_instances, lego_printer=aws_stop_instances_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to run filter untagged EC2 instances and stop those instances. To view the full platform capabilities of unSkript please visit https://unskript.com